In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Separando as features e a label
df = pd.read_csv("train_w_metricas.csv",index_col=0)
y = X['label']
columns_to_remove = [
    'label', 'code', 'cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 
    'dit', 'noc', 'rfc', 'lcom', 'lcom*', 'tcc', 'lcc'
]
X = df.drop(columns=columns_to_remove, axis=1)

In [3]:
# Aplicando o SMOTE
smote = SMOTE(random_state=None)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [4]:
# Dividir o conjunto de dados
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=None)


In [5]:
param_grid = {
    'criterion': ['gini', 'entropy'],          # Função de avaliação
    'splitter': ['best', 'random'],            # Método de divisão dos nós
    'max_depth': [None, 10, 20, 30, 40, 50],   # Profundidade máxima da árvore
    'min_samples_split': [2, 10, 20],          # Mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 5, 10],            # Mínimo de amostras em uma folha
    'max_features': [None, 'sqrt', 'log2'],    # Máximo de features consideradas na divisão
    'class_weight': [None, 'balanced']         # Adiciona a opção de balanceamento
}
dt = DecisionTreeClassifier(random_state=None)
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5,scoring='accuracy',n_jobs=-1,verbose=1)


In [ ]:
# Ajustar o GridSearch nos dados de treino
grid_search.fit(X_train, y_train)

print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)


# melhor modelo encontrado
melhor_modelo = grid_search.best_estimator_
y_pred = melhor_modelo.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Melhores parâmetros encontrados:
{'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}
Accuracy: 0.6976744186046512
Precision: 0.7341772151898734
Recall: 0.6976744186046512
F1-Score: 0.6972654283700795
